In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install -q fastapi==0.68.1
!pip install -q python-multipart==0.0.5




In [ ]:
!pip install -q python-multipart==0.0.5

In [ ]:
!pip install -q uvicorn==0.15.0

In [ ]:
!pip install -q nest-asyncio

In [ ]:
!pip install -q pyngrok

In [ ]:
!pip install transformers

In [ ]:
!pip install kaleido python-multipart uvicorn cohere openai tiktoken

In [10]:
import os
import requests

from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn

import torch
import math
import pandas as pd
from geopy import Point
from geopy.distance import distance
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm_notebook

In [11]:
import numpy as np
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer, util

In [12]:
torch.manual_seed(42)

In [13]:
var = "all-mpnet-base-v2"
model = SentenceTransformer(var)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
CRIME_DATA = '/content/drive/MyDrive/DivaGuard/crime_data.csv'
POLICE_DATA = '/content/drive/MyDrive/DivaGuard/police_station_lat_log_data.csv'

In [15]:
def get_routes(start, dest):
    KEY = 'STgdB26jzTqnE2YDF11UFBDYzvj4G70L'
    URL = f'https://www.mapquestapi.com/directions/v2/alternateroutes?key={KEY}'

    payload = {
        "locations": [start, dest],
        "maxRoutes": 6,
        "timeOverage": 100
    }
    response = requests.post(URL, json=payload)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch route data from MapQuest API")
        return None

In [16]:
def process_data(data):
  node_dict = {}
  nodes = []
  for node in data['route']['legs'][0]['maneuvers']:
    nodes.append(node['startPoint'])
    nodes[-1]['dist'] = node['distance']
    nodes[-1]['narr'] = node['narrative']
  node_dict[0] = nodes
  num_alternates = len(data['route']['alternateRoutes'])
  for i in range(num_alternates):
    nodes = []
    r = data['route']['alternateRoutes'][i]['route']['legs'][0]
    num_manuvs = len(r['maneuvers'])
    for j in range(num_manuvs):
      nodes.append(r['maneuvers'][j]['startPoint'])
      nodes[-1]['dist'] = r['maneuvers'][j]['distance']
      nodes[-1]['narr'] = r['maneuvers'][j]['narrative']
    node_dict[i+1] = nodes
  return node_dict

In [17]:
def get_km_range(lat, lng, k):
  p0 = Point(lat, lng)
  p1 = distance(kilometers=k).destination(point=p0, bearing=0)
  p2 = distance(kilometers=k).destination(point=p0, bearing=90)
  lat_range = p1.latitude - p0.latitude
  lng_range = p2.longitude - p0.longitude
  return lat_range, lng_range

def get_distance(xa, ya, xb, yb):
  return math.sqrt((xa-xb)**2 + (ya-yb)**2)

In [18]:
def get_severity(category):
  weights = {
      'Poor / No Street Lighting': 1,
      'Stalking': 2,
      'Ogling/Facial Expressions/Staring': 2,
      'Commenting': 3,
      'Catcalls/Whistles': 3,
      'Taking pictures': 4,
      'Indecent exposure': 5,
      'Sexual Invites': 5,
      'Chain Snatching / Robbery': 8,
      'Touching /Groping': 8,
      'Rape / Sexual Assault': 10,
  }
  return weights[category]

In [19]:
def get_risk_factor(routes):
  risk_scores = []
  df = pd.read_csv('/content/drive/MyDrive/DivaGuard/crime_data.csv')
  for idx in routes.keys():
    route_risk = 0
    for node in routes[idx]:
      lat, lng = node['lat'], node['lng']
      dist = node['dist']
      if dist==0: dist=1
      lat_r, lng_r = get_km_range(lat, lng, 2)
      crimes = df.loc[
          (df['Latitude'].between(lat-lat_r, lat+lat_r)) &
          (df['Longitude'].between(lng-lng_r, lng+lng_r))]
      # route_risk += crimes.shape[0]
      node_risk = 0
      for id, crime in crimes.iterrows():
        node_risk += get_severity(crime['Category'])*get_distance(
            crime['Latitude'], crime['Longitude'], lat, lng)
      route_risk += (node_risk/dist)

    risk_scores.append(route_risk)
  return risk_scores

In [20]:
def find_safest_path(routes, scores):
    # Find the route with the lowest risk score
    min_score_index = scores.index(min(scores))
    safest_path = routes[min_score_index]
    return safest_path

In [21]:
app = FastAPI()

class RoutePlanner(BaseModel):
    start_loc: str
    end_loc: str

class CrimeReportData(BaseModel):
    location: str
    incident: str


@app.post("/routes")
async def alternate_routes(inp_data: RoutePlanner):
  data = get_routes(inp_data.start_loc, inp_data.end_loc)
  node_dict = process_data(data)
  scores = get_risk_factor(node_dict)
  max_score = max(scores)
  scores = [i / max_score for i in scores]
  return {"routes": node_dict, "risk_scores": scores}

In [22]:
@app.post("/add_data")
async def add_to_dataframe(inp_data: CrimeReportData):
  geolocator = Nominatim(timeout=10, user_agent="my-app")
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
  coord = geocode(inp_data.location)
  df = pd.read_csv('/content/drive/MyDrive/DivaGuard/crime_data.csv')
  model = SentenceTransformer(var)
  with torch.no_grad():
      category = model(inp_data.incident)
  df = df.append({"Category":str(category), "Latitude":coord.latitude, "Longitude":coord.longitude,"Incident":str(inp_data.incident), "Location":str(inp_data.location)}, ignore_index=True)

In [23]:
@app.post("/safest_path")
async def safest_path(inp_data: RoutePlanner):
    data = get_routes(inp_data.start_loc, inp_data.end_loc)
    if data:
        node_dict = process_data(data)
        scores = get_risk_factor(node_dict)
        safest_route = find_safest_path(node_dict, scores)
        return {"safest_route": safest_route}
    else:
        return {"error": "Failed to fetch route data from MapQuest API"}

In [31]:
auth_token = "2O8XV6yH8IEkUvZ77UC2nZpK1OP_555Xi375PfZLjk8ks1yAS"
os.system(f"ngrok authtoken {auth_token}")


0

In [32]:
!ps aux | grep ngrok

root        3418  0.0  0.0   7376  3516 ?        S    18:18   0:00 /bin/bash -c ps aux | grep ngrok
root        3420  0.0  0.0   6484  2276 ?        S    18:18   0:00 grep ngrok


In [33]:
public_url = ngrok.connect(8000, bind_tls=True)


In [ ]:
nest_asyncio.apply()  # allow for asyncio to work within the Jupyter notebook cell
print(public_url)     # public URL for the server via a tunnel
uvicorn.run(app)      # run the FastAPI app using uvicorn

NgrokTunnel: "https://2e77-34-136-211-1.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1499]
INFO:uvicorn.error:Started server process [1499]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2409:40e5:1017:605f:d071:93d1:5abc:f3e2:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:40e5:1017:605f:d071:93d1:5abc:f3e2:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:40e5:1017:605f:d071:93d1:5abc:f3e2:0 - "GET /images/icons/gear.png HTTP/1.1" 404 Not Found
INFO:     152.59.132.4:0 - "POST /safest_path HTTP/1.1" 200 OK
INFO:     103.15.228.94:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.15.228.94:0 - "GET /images/icons/gear.png HTTP/1.1" 404 Not Found
INFO:     103.15.228.94:0 - "POST /safest_path HTTP/1.1" 200 OK
INFO:     152.59.135.29:0 - "POST /safest_path HTTP/1.1" 200 OK
